# Segmenting and Clustering Neighborhoods in Toronto

This notebook has included all three questions in the following sections

## Question 1 scrape the data from wiki and create into a pandas dataframe

### import the relevant modules

In [1]:
import pandas as pd
import requests

scrape the data from wiki and read the data

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_raw = requests.get(url)
toronto_data = pd.read_html(toronto_raw.text)
toronto_data

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

In [15]:
toronto_data = toronto_data[0]
toronto_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Remove the rows with borough that is not assigned

In [28]:
toronto = toronto_data[toronto_data['Borough'] != 'Not assigned']
toronto.reset_index(drop='index')

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


check if the Postal Code are unique or not

In [22]:
toronto['Postal Code'].nunique()

103

check if there is any Neigbourhood that is not assigned

In [29]:
toronto.Neighbourhood.str.count("Not assigned").sum()

0

In [30]:
print("The number of rows of toronto DataFrame is {}".format(toronto.shape[0]))

The number of rows of toronto DataFrame is 103


## Question 2 Add latitude and longitude into the dataframe

In [33]:
!pip install geocoder
import geocoder

print("geocoder has been imported")

     |████████████████████████████████| 98 kB 10.3 MB/s eta 0:00:01
geocoder has been imported


In [34]:
ll_data = pd.read_csv('https://cocl.us/Geospatial_data')
ll_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


The postcode is exactly the same with toronto dataframe, so we can join the two dataframe together

In [35]:
toronto_fixed = toronto.join(ll_data.set_index('Postal Code'), on='Postal Code', how='inner')

In [36]:
toronto_fixed

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Question 3 Explore and Cluster neighborhoods in Toronto

In [38]:
from geopy.geocoders import Nominatim
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

     |████████████████████████████████| 94 kB 6.6 MB/s  eta 0:00:01


In [41]:
toronto_fixed['Borough'].values

array(['North York', 'North York', 'Downtown Toronto', 'North York',
       'Downtown Toronto', 'Etobicoke', 'Scarborough', 'North York',
       'East York', 'Downtown Toronto', 'North York', 'Etobicoke',
       'Scarborough', 'North York', 'East York', 'Downtown Toronto',
       'York', 'Etobicoke', 'Scarborough', 'East Toronto',
       'Downtown Toronto', 'York', 'Scarborough', 'East York',
       'Downtown Toronto', 'Downtown Toronto', 'Scarborough',
       'North York', 'North York', 'East York', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East York', 'Downtown Toronto', 'West Toronto', 'Scarborough',
       'North York', 'North York', 'East Toronto', 'Downtown Toronto',
       'West Toronto', 'Scarborough', 'North York', 'North York',
       'East Toronto', 'Downtown Toronto', 'North York', 'North York',
       'Scarborough', 'North York', 'North York', 'East Toronto',
       'North York', 'York', 'North York', 'Scarborough', 'Nort

In [51]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto with neighborhoos

In [52]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_fixed['Latitude'], toronto_fixed['Longitude'], toronto_fixed['Borough'], toronto_fixed['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [75]:
central_toronto = toronto_fixed[toronto_fixed['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678


get the geographical coordinates of east toronto

In [76]:
address = 'Central Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [78]:
# create map of East Toronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

### Define Foursquare Credentials and Version

In [58]:
CLIENT_ID = '1HSGUO3ID1CT4OO11SBFWQII00YVCOYQNK30JQE2YRJUI3HS' # Foursquare ID
CLIENT_SECRET = '30JPXZB3DO45HCTEJFXIQRYP5FOBBU24PQRFBSJTYSYLXVPC' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1HSGUO3ID1CT4OO11SBFWQII00YVCOYQNK30JQE2YRJUI3HS
CLIENT_SECRET:30JPXZB3DO45HCTEJFXIQRYP5FOBBU24PQRFBSJTYSYLXVPC


#### Let's explore the first neighborhood in our dataframe.

In [79]:
central_toronto.loc[0, 'Neighbourhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [80]:
neighborhood_latitude = central_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Now, let's get the top 50 venues that are in Marble Hill within a radius of 500 meters.

In [81]:
limit = 50
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        neighborhood_latitude,
        neighborhood_longitude,
        radius,
        limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=1HSGUO3ID1CT4OO11SBFWQII00YVCOYQNK30JQE2YRJUI3HS&client_secret=30JPXZB3DO45HCTEJFXIQRYP5FOBBU24PQRFBSJTYSYLXVPC&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=50'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601664f5df0a976dd3225334'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.72696303913755,
          'lng': -79.39438246708775}],
        'distance': 465,
        'c

In [83]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [84]:
from pandas.io.json import json_normalize 

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,HYC Design Inc.,Business Service,43.726793,-79.391681
2,Zodiac Swim School,Swim School,43.728532,-79.382860
3,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [85]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [103]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [106]:
central_toronto_venues = getNearbyVenues(names = central_toronto['Neighbourhood'],
                                  latitudes = central_toronto['Latitude'],
                                  longitudes = central_toronto['Longitude'])

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


#### Let's check the size of the resulting dataframe

In [107]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(108, 7)


,Neighbourhood,Neighboirhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,HYC Design Inc.,43.726793,-79.391681,Business Service
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden


check how many venues were returned for each neighborhood

In [108]:
central_toronto_venues.groupby('Neighbourhood').count()

,Neighboirhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,8,8,8,8,8,8
"Forest Hill North & West, Forest Hill Road Park",4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",3,3,3,3,3,3
"North Toronto West, Lawrence Park",18,18,18,18,18,18
Roselawn,2,2,2,2,2,2
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",14,14,14,14,14,14
"The Annex, North Midtown, Yorkville",20,20,20,20,20,20


#### Let's find out how many unique categories can be curated from all the returned venues

In [109]:
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 61 uniques categories.


### Analyze each Neighbourhood

In [110]:
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighbourhood'] = central_toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,...,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
central_toronto_onehot.shape

(108, 62)

#### group rows by neighborhood to get the mean of the frequency of occurrence of each category

In [112]:
central_toronto_grouped = central_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
central_toronto_grouped

,Neighbourhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Breakfast Spot,Brewery,Burger Joint,Bus Line,Business Service,...,Spa,Sporting Goods Shop,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.00,0.000000,0.000000,0.000,0.028571,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.057143,0.00,0.028571,0.028571,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.00,0.000000,0.000000,0.125,0.000000,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Forest Hill North & West, Forest Hill Road Park",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000
3,Lawrence Park,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.25,0.25,...,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000
5,"North Toronto West, Lawrence Park",0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.00,...,0.055556,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.055556
6,Roselawn,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.071429,0.00,0.071429,0.071429,0.000,0.000000,0.00,0.00,0.00,...,0.000000,0.000000,0.071429,0.071429,0.00,0.000000,0.000000,0.000000,0.071429,0.000000
8,"The Annex, North Midtown, Yorkville",0.000000,0.05,0.000000,0.000000,0.000,0.000000,0.05,0.00,0.00,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


#### convert it into pandas dataframe

create a function to sort the venues in descending order.

In [113]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create the new dataframe and display the top 10 venues for each neighborhood.

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = central_toronto_grouped['Neighbourhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Italian Restaurant,Gym,Café,Coffee Shop,Greek Restaurant,Diner
1,Davisville North,Gym / Fitness Center,Food & Drink Shop,Dance Studio,Sandwich Place,Park,Department Store,Breakfast Spot,Hotel,Furniture / Home Store,Fried Chicken Joint
2,"Forest Hill North & West, Forest Hill Road Park",Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Yoga Studio
3,Lawrence Park,Park,Bus Line,Swim School,Business Service,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Yoga Studio
4,"Moore Park, Summerhill East",Park,Trail,Playground,Farmers Market,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Yoga Studio


### Run k-means to cluster Neighborhoods into 3 clusters

In [129]:
# set number of clusters
kclusters = 3

central_toronto_grouped_clustering = central_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 2, 0, 1, 0, 0], dtype=int32)

add the cluster label into the dataframe

In [131]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_toronto_merged = central_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
central_toronto_merged = central_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

central_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Business Service,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Yoga Studio
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Music Venue,Yoga Studio,Department Store,Gourmet Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Gym / Fitness Center,Food & Drink Shop,Dance Studio,Sandwich Place,Park,Department Store,Breakfast Spot,Hotel,Furniture / Home Store,Fried Chicken Joint
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,2,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Yoga Studio
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Salon / Barbershop,Furniture / Home Store,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant


### Create the map and visualize the clusters

In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_toronto_merged['Latitude'], central_toronto_merged['Longitude'], central_toronto_merged['Neighbourhood'], central_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1

In [133]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 0, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0,Park,Bus Line,Swim School,Business Service,Dessert Shop,Diner,Discount Store,Donut Shop,Farmers Market,Yoga Studio
2,Central Toronto,0,Gym / Fitness Center,Food & Drink Shop,Dance Studio,Sandwich Place,Park,Department Store,Breakfast Spot,Hotel,Furniture / Home Store,Fried Chicken Joint
4,Central Toronto,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Salon / Barbershop,Furniture / Home Store,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Chinese Restaurant
5,Central Toronto,0,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Park,Pizza Place,Pub,Indian Restaurant,Liquor Store,Pharmacy
6,Central Toronto,0,Dessert Shop,Sandwich Place,Sushi Restaurant,Pizza Place,Italian Restaurant,Gym,Café,Coffee Shop,Greek Restaurant,Diner
8,Central Toronto,0,Coffee Shop,American Restaurant,Vietnamese Restaurant,Light Rail Station,Pizza Place,Pub,Restaurant,Liquor Store,Supermarket,Sushi Restaurant


### Cluster 2

In [134]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 1, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,1,Garden,Music Venue,Yoga Studio,Department Store,Gourmet Shop,Gas Station,Furniture / Home Store,Fried Chicken Joint,Food & Drink Shop,Fast Food Restaurant


### Cluster 3

In [135]:
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 2, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,2,Park,Jewelry Store,Trail,Sushi Restaurant,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Yoga Studio
7,Central Toronto,2,Park,Trail,Playground,Farmers Market,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Yoga Studio
